In [ ]:
export ZONE=$(gcloud compute project-info describe \
--format="value(commonInstanceMetadata.items[google-compute-default-zone])")

##Task 1. Deploy a GKE cluster

In [ ]:
gcloud container clusters create gmp-cluster --num-nodes=3 --zone=$ZONE

In [ ]:
gcloud container clusters get-credentials gmp-cluster --zone=$ZONE

##Task 2. Deploy a managed collection

##Task 3. Deploy an example application

##Task 4. Filter exported metrics

In [ ]:
kubectl create ns gmp-test

In [ ]:
kubectl -n gmp-test apply -f https://raw.githubusercontent.com/GoogleCloudPlatform/prometheus-engine/v0.2.3/manifests/setup.yaml

In [ ]:
kubectl -n gmp-test apply -f https://raw.githubusercontent.com/GoogleCloudPlatform/prometheus-engine/v0.2.3/manifests/operator.yaml

In [ ]:
kubectl -n gmp-test apply -f https://raw.githubusercontent.com/GoogleCloudPlatform/prometheus-engine/v0.2.3/examples/example-app.yaml

In [ ]:
cat > op-config.yaml <<'EOF_END'
apiVersion: monitoring.googleapis.com/v1alpha1
collection:
  filter:
    matchOneOf:
    - '{job="prom-example"}'
    - '{__name__=~"job:.+"}'
kind: OperatorConfig
metadata:
  annotations:
    components.gke.io/layer: addon
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"monitoring.googleapis.com/v1alpha1","kind":"OperatorConfig","metadata":{"annotations":{"components.gke.io/layer":"addon"},"labels":{"addonmanager.kubernetes.io/mode":"Reconcile"},"name":"config","namespace":"gmp-public"}}
  creationTimestamp: "2022-03-14T22:34:23Z"
  generation: 1
  labels:
    addonmanager.kubernetes.io/mode: Reconcile
  name: config
  namespace: gmp-public
  resourceVersion: "2882"
  uid: 4ad23359-efeb-42bb-b689-045bd704f295
EOF_END

In [ ]:
export PROJECT=$(gcloud config get-value project)

In [ ]:
gsutil mb -p $PROJECT gs://$PROJECT

In [ ]:
gsutil cp op-config.yaml gs://$PROJECT

In [ ]:
gsutil -m acl set -R -a public-read gs://$PROJECT